In [7]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [8]:
# variables

data_dir = "./data/"
print(f"Loading PDFs from: {data_dir}")

Loading PDFs from: ./data/


In [9]:
""" data ingestion

1. read all pdfs in data_dir
2. 
3. 

"""

' data ingestion\n\n1. read all pdfs in data_dir\n2. \n3. \n\n'

In [10]:
# import fitz  # PyMuPDF




# doc = fitz.open("/home/fuths/projects/graphrag_weaviate_1/data/Beyond Backpropagation Exploring Innovative Algorithms for.pdf")

# for page_num, page in enumerate(doc, start=1):
#     text = page.get_text("text")  # modes: "text", "blocks", "dict", "json", "html"
#     print(f"--- Page {page_num} ---")
#     print(text)



In [11]:
from langchain_community.document_loaders import PyMuPDFLoader

file_path = "/home/fuths/projects/graphrag_weaviate_1/data/Beyond Backpropagation Exploring Innovative Algorithms for.pdf"
loader = PyMuPDFLoader(file_path)

In [12]:
docs = loader.load()


In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 200 sub-documents.


In [14]:
# for i in all_splits:
#     print(i)

In [15]:
# embedding models

# embedding model

from langchain_ollama import OllamaEmbeddings


embedding_model = OllamaEmbeddings(model="nomic-embed-text")

In [16]:
# vectordb

import weaviate
from langchain_weaviate.vectorstores import WeaviateVectorStore

weaviate_client = weaviate.connect_to_local()

vectorstore = WeaviateVectorStore.from_documents(
    client=weaviate_client,
    documents=all_splits,
    embedding=embedding_model,
    index_name="graphrag_v2_test1",  # name your class
)

# retriver = vectorstore.as_retriever(search_type="mmr")  # max marginal relevance

In [17]:
query = "what training methods are investigated?"
docs = vectorstore.similarity_search(query)

# Print the first 100 characters of each result
for i, doc in enumerate(docs):
    print(f"\nDocument {i + 1}:")
    print(doc.page_content[:100] + "...")


Document 1:
complexity (MNIST, Fashion-MNIST, CIFAR-10, and CIFAR-100), the scalability and
practical applicabil...

Document 2:
complexity (MNIST, Fashion-MNIST, CIFAR-10, and CIFAR-100), the scalability and
practical applicabil...

Document 3:
versions or fixed training durations, the author conducted all experiments within a consistent,
mode...

Document 4:
versions or fixed training durations, the author conducted all experiments within a consistent,
mode...


In [18]:
retriever = vectorstore.as_retriever(search_type="mmr")

In [19]:
retriever.invoke(query)[0].page_content

'versions or fixed training durations, the author conducted all experiments within a consistent,\nmodern software environment (detailed in Section 3.8) and used early stopping to determine\nthe effective training duration, thus ensuring an equitable comparison of algorithms on contem-\nporary hardware.\n18'

In [20]:
# qa models

from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2:latest",
    temperature=0,
    # other params...
)

In [21]:
messages = [
    (
        "system",
        "You are a helpful assistant that answers the following question.",
    ),
    ("human", "what is programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='Programming is the process of designing, writing, testing, and maintaining the instructions that a computer follows to perform a specific task or set of tasks. These instructions, known as "code," are written in one or more programming languages, which are used to communicate with computers.\n\nProgramming involves using a combination of logic, problem-solving skills, and creativity to write code that can be executed by a computer. The ultimate goal of programming is to create software, apps, websites, games, and other digital products that can perform specific tasks or provide useful services to users.\n\nSome common aspects of programming include:\n\n1. **Writing code**: Using a programming language to write instructions that the computer can understand.\n2. **Problem-solving**: Identifying problems and developing solutions using code.\n3. **Debugging**: Finding and fixing errors in the code to ensure it works correctly.\n4. **Testing**: Verifying that the code pro

In [22]:
# cypher models



In [23]:
print(len(all_splits))

200


In [24]:
# graph db

from langchain_neo4j import Neo4jGraph

graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="test1234")

In [ ]:
# testing out custom note relation extraction

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

prompt = PromptTemplate(
    template="""You are an expert at extracting nodes and relations from documents. 
    you have to extract nodes and relations from the document provided.
    the nodes should be the key concepts, entities, or ideas present in the document.
    provide them with appropriate labels.
    the relations should describe how these nodes are connected or related to each other.

    try and reuse labels for nodes wherever possible.
    provide the output in json format 
    
    Here is the document: 
    {document}

    """,
    input_variables=["document"],
)


node_relation_extractor = prompt | llm | JsonOutputParser()


In [48]:
"""
  the nodes should be the key concepts, entities, or ideas present in the document.
provide them with appropriate labels.
  the relations should describe how these nodes are connected or related to each other.




  provide the output in json format
as shown below:
    {{
        "nodes": [
            {{"id": "node1", "label": "Node Label 1", "properties": {{"property1": "value1", "property2": "value2"}}}},
            {{"id": "node2", "label": "Node Label 2", "properties": {{"property1": "value1", "property2": "value2"}}}}
        ],
        "relations": [
            {{"from": "node1", "to": "node2", "type": "RELATION_TYPE", "properties": {{"property1": "value1"}}}}
        ]
    }}
"""

'\n  the nodes should be the key concepts, entities, or ideas present in the document.\nprovide them with appropriate labels.\n  the relations should describe how these nodes are connected or related to each other.\n\n\n\n\n  provide the output in json format\nas shown below:\n    {{\n        "nodes": [\n            {{"id": "node1", "label": "Node Label 1", "properties": {{"property1": "value1", "property2": "value2"}}}},\n            {{"id": "node2", "label": "Node Label 2", "properties": {{"property1": "value1", "property2": "value2"}}}}\n        ],\n        "relations": [\n            {{"from": "node1", "to": "node2", "type": "RELATION_TYPE", "properties": {{"property1": "value1"}}}}\n        ]\n    }}\n'

In [51]:
print(len(all_splits))
print(all_splits[1].page_content)


docs = node_relation_extractor.invoke(all_splits[1].page_content)
print(len(docs))
print(docs,'\n')



200
To ensure a fair and insightful analysis, a robust comparative framework was established.
Each alternative algorithm was implemented on its native architecture (MLPs for FF and
MF, a CNN for CaFo) and systematically compared against an equivalent model trained
with BP. Critically, this methodology mandates comprehensive hyperparameter optimiza-
tion for all algorithms using Optuna, coupled with consistent early stopping criteria based
on validation performance, thereby ensuring that all comparisons are conducted between
optimally tuned models.
The investigation culminates in a significant finding: The Mono-Forward (MF) algo-
rithm not only competes with, but consistently surpasses backpropagation in classification
accuracy on its native MLP architectures. This superior generalization arises from the al-
gorithm’s ability to converge to a more favorable minimum in the validation loss landscape,


TypeError: object of type 'AIMessage' has no len()

In [46]:
print(docs,'\n')

{'nodes': [{'id': '1', 'label': 'Algorithm'}, {'id': '2', 'label': 'Model'}, {'id': '3', 'label': 'Hyperparameter Optimization'}, {'id': '4', 'label': 'Early Stopping Criteria'}, {'id': '5', 'label': 'Validation Performance'}, {'id': '6', 'label': 'Optimization Methodology'}, {'id': '7', 'label': 'Mono-Forward Algorithm'}, {'id': '8', 'label': 'Backpropagation'}], 'relations': [{'source': '1', 'target': '2', 'relation': 'Implemented On'}, {'source': '3', 'target': '5', 'relation': 'Optimized For'}, {'source': '4', 'target': '5', 'relation': 'Based On'}, {'source': '6', 'target': '1,2,7,8', 'relation': 'Compared Against'}, {'source': '7', 'target': '8', 'relation': 'Competes With'}, {'source': '7', 'target': '5', 'relation': 'Surpasses In Classification Accuracy'}]} 



In [27]:
# # add nodes and relations to the graph database

# from langchain_experimental.graph_transformers import LLMGraphTransformer
# import time

# graph_transformer = LLMGraphTransformer(
#     llm=llm,
#     # allowed_nodes=["Concept"],
#     # node_properties=[""],
#     # allowed_relationships=["AUTHORED", "", "RELATED_TO"],
# )
# print("1")

# graph_documents = []
# for i,doc in enumerate(all_splits):
#     stime = time.time()
#     # Process one document at a time
#     graph_doc = graph_transformer.convert_to_graph_documents([doc])
#     graph_documents.extend(graph_doc)
#     print(f"end of chunk, time taken: {time.time() - stime}")

In [28]:
# # querying graphdb

# from langchain_neo4j import GraphCypherQAChain

# chain = GraphCypherQAChain.from_llm(
#     llm=llm,
#     graph=graph,
#     verbose=True,
#     return_intermediate_steps=True,
#     allow_dangerous_requests=True,
# )

# chain.invoke(query)